In [ ]:
import scimap as sm
import pandas as pd
import sys
import os
import scanpy as sc
import seaborn as sns; sns.set(color_codes=True)
import anndata
import matplotlib.pyplot as plt
import numpy as np
adata = anndata.read_h5ad(path_to_adata_file)
adata.obs

In [ ]:
# Running spatial expression calculation
adata = sm.tl.spatial_expression(adata, x_coordinate='X_centroid',
                                 y_coordinate='Y_centroid',
                                 method='knn', knn=30, imageid='imageid', 
                                 use_raw=True, subset=None,
                                 label='spatial_expression_knn30')

In [ ]:
# Running spatial clustering
adata = sm.tl.spatial_cluster(adata, k=11, df_name='spatial_expression_knn30', method='kmeans', label='spatial_kmeans_knn30_k11')

In [ ]:
#Marker expression per unsupervised niche cluster. The heatmap is available on github...
sc.pl.matrixplot(adata, var_names= adata.var.index, groupby='spatial_kmeans_knn30_k11', dendrogram=False, use_raw=False, cmap="vlag", standard_scale='var')

In [ ]:
# Visualizing niches in representative tissue sections. The output is available on github...

unique_clusters = adata.obs['spatial_kmeans_knn30_k11'].unique()
num_clusters = len(unique_clusters)

# List of sample names
sample_names = ['H3', 'H2a', 'P1a', 'P9a']

# Define rotations for each sample
rotations = {
    'H3': 180,
    'H2a': 250,
    'P1a': 230,
    'P9a': 180
}

# Generate plots for each cluster
for cluster in unique_clusters:
    fig = plt.figure(figsize=(12, 12))
    fig.patch.set_facecolor('black')

    # Loop over each sample to generate spatial scatter plots
    for idx, sample_name in enumerate(sample_names):
        # Subset the data for the current sample
        subset_adata = adata[adata.obs['imageid'] == sample_name]
        
        # Get the coordinates
        x_coords = subset_adata.obs['X_centroid']
        y_coords = subset_adata.obs['Y_centroid']
        
        # Apply rotation
        angle = np.deg2rad(rotations[sample_name])
        new_x_coords = x_coords * np.cos(angle) - y_coords * np.sin(angle)
        new_y_coords = x_coords * np.sin(angle) + y_coords * np.cos(angle)
        
        # Calculate the aspect ratio
        x_range = new_x_coords.max() - new_x_coords.min()
        y_range = new_y_coords.max() - new_y_coords.min()
        aspect_ratio = x_range / y_range

        # Determine subplot position
        ax = fig.add_axes([
            (idx % 2) * 0.5,  # x position
            (1 - idx // 2) * 0.5,  # y position
            0.5 * aspect_ratio,  # width
            0.5  # height
        ])
        
        # Set black background for the subplot
        ax.set_facecolor('black')
        
        # Remove the outline
        for spine in ax.spines.values():
            spine.set_visible(False)
        
        # Map colors such that the current cluster is white and others are grey
        color_mapping = subset_adata.obs['spatial_kmeans_knn30_k11'].apply(
            lambda x: 'white' if x == cluster else 'blue'
        )
        
        scatter = ax.scatter(
            x=new_x_coords, 
            y=new_y_coords, 
            c=color_mapping, 
            s=0.3  # Slightly smaller dot size
        )
        
        # Add the sample name below the plot
        ax.text(
            0.5, 1, sample_name, 
            horizontalalignment='center', 
            verticalalignment='center', 
            transform=ax.transAxes, 
            color='white', fontsize=12, weight='bold'
        )
        
        # Remove grids
        ax.grid(False)
        
        # Remove ticks and their labels
        ax.set_xticks([])
        ax.set_yticks([])

    # Add the cluster label to the overall figure, higher and with bigger font
    plt.suptitle(cluster, color='white', fontsize=20, weight='bold', x=0.4, y=1.1)
    
    plt.show()

In [ ]:
# Rename unsupervised niche clusters to reflect the histologic landmarks
adata.obs['niche_renamed'] = adata.obs['spatial_kmeans_knn30_k11']

# Define the replacement dictionary
replacement_dict = {
    '8': 'Im-Str.2',
    '3': 'Im-Str.1',
    '2': 'Plasma',
    '0': 'FibCT',
    '4': 'OE.B-PB',
    '10': 'BV',
    '5': 'Epi-CT',
    '9': 'T-B-APC',
    '1': 'OE.Sp-K',
    '7': 'NeutCT',
    '6': 'TAE',
}

# Replace values in the 'niche_renamed' column
adata.obs['niche_renamed'] = adata.obs['niche_renamed'].replace(replacement_dict)

# Print unique values after replacement
print(adata.obs['niche_renamed'].unique())

In [ ]:
#Merging the 'OE.B-PB' and 'OE-Sp-Krt' into OE (oral epithelial niche)
adata.obs['niche_merged'] = adata.obs['niche_renamed']

# Define the replacement dictionary
replacement_dict = {
    'OE.B-PB': 'OE',
    'OE.Sp-K': 'OE'
}

# Replace values in the 'lvl3_spatial_cluster' column
adata.obs['niche_merged'] = adata.obs['niche_merged'].replace(replacement_dict)

# Print unique values after replacement
print(adata.obs['niche_merged'].unique())